## Food Bank Problem

### Algorithm 1 - OPT Offline

In [4]:
import sys
import numpy as np

In [5]:
num_iterations = 0
num_towns = 5
town_demands = np.zeros(num_towns)
town_expected_demands = np.zeros(num_towns)
for i in range(num_towns):
    town_max = np.random.uniform(0,10)
    town_demands[i] = np.random.uniform(0, town_max)
    town_expected_demands[i] = town_max/2
budget = 5

In [6]:
## Water-filling Algorithm for sorted demands
def waterfilling_sorted(d,b):
    n = np.size(d)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        equal_allocation = bundle_remaining/(n-i)
        if d[i]<equal_allocation:
            allocations[i] = bundle_remaining if i==n-1 else d[i]
        else:
            allocations[i] = equal_allocation
        bundle_remaining -= allocations[i]
    return allocations

In [9]:
## Water-filling Algorithm for general demands
def waterfilling(d,b):
    n = np.size(d)
    sorted_indices = np.argsort(d)
    sorted_demands = np.sort(d)
    sorted_allocations = waterfilling_sorted(sorted_demands, b)
    allocations = np.zeros(n)
    for i in range(n):
        allocations[sorted_indices[i]] = sorted_allocations[i]
    return allocations

In [15]:
## Tests
assert list(waterfilling(np.zeros(0), 5)) == []
assert list(waterfilling(np.array([1,2,3,4]), 10)) == [1,2,3,4]
assert list(waterfilling(np.array([3,4,1,2]), 10)) == [3,4,1,2]
assert list(waterfilling(np.array([1,2,3,4]), 8)) == [1,2,2.5,2.5]
assert list(waterfilling(np.array([3,1,4,2]), 8)) == [2.5,1,2.5,2]
assert list(waterfilling(np.array([3,6,5,6]), 8)) == [2,2,2,2]

In [22]:
## Online Water-filling taking minimum of realized demand and predetermeined allocation
def waterfilling_online_1(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    prior_allocations_assignment = waterfilling(demands_predicted,b)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = min(prior_allocations_assignment[i], demands_realized[i]) if i!=n-1 else bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [24]:
## Tests
assert list(waterfilling_online_1(np.zeros(0), np.zeros(0), 5)) == []
assert list(waterfilling_online_1(np.array([1,2,3,4]), np.array([5,5,5,5]), 10)) == [1,2,3,4]
assert list(waterfilling_online_1(np.array([3,1,4,2]), np.array([2,3,2.5,1]), 8)) == [2,1,2.5,2.5]


In [16]:
## Online Water-filling algorithm where each agent solves waterfilling with realized current demand and expected following demands
def waterfilling_online_2(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = waterfilling(np.append(demands_realized[i],demands_predicted[i+1:]), bundle_remaining)[0]
        bundle_remaining -= allocations[i]
    return allocations

In [ ]:
## Online Water-filling algorithm where each agent is assigned infinite demand while finding optimal solution and allocation is readjusted
def waterfilling_online_3(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        future_allocations = waterfilling(np.append(np.Inf,demands_predicted[i+1:]), bundle_remaining)
        if future_allocations[0]>demands_realized[i]:
            allocations[i] = demands_realized[i]
        else:
            allocations[i] = future_allocations[0]
        bundle_remaining -= allocations[i]
    return allocations

In [ ]:
## Calculate log of Nash welfare for objective function
def objective(demands, allocation):
    welfare_sum = 0
    for i in range(np.size(demands)):
        welfare_sum += np.log(np.min(1,allocation[i]/demands[i]))
    return welfare_sum

In [19]:
print(waterfilling_sorted(np.sort(town_demands), budget))

[0.05142401 0.43414925 0.65527242 0.92986868 2.92928564]


### Algorithm 2

### Algorithm 3?